No cancellation fees. More expensive on weekends

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('../data/silvercar_reservations.csv')
df_users = pd.read_csv('../data/silvercar_users.csv')
df_promos = pd.read_csv('../data/silvercar_promotions.csv', encoding="ISO-8859-1")
df_promo_codes = pd.read_csv('../data/silvercar_promo_codes.csv', encoding="ISO-8859-1")
df_locs = pd.read_csv('../data/silvercar_locations.csv', encoding="ISO-8859-1")

In [3]:
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df.head()

,id,pickup,dropoff,user_id,status,confirmation_token,created_at,updated_at,booked_price,current_state,...,publisher_id,cancelled_at,local_rental,booked_by_id,awards_referral_bonus,extole_conversion_id,potential_local_rental,claimed_at,customer_modifiable,credit_card_id
0,89,41288.29,41289.40,12.00,nan,5B23231DC0,41269.81,41417.80,nan,cancelled,...,NaN,nan,0,nan,0,nan,nan,nan,1,nan
1,91,41307.79,41308.79,49.00,nan,9A10C66F0C,41269.85,41417.80,nan,cancelled,...,NaN,nan,0,nan,0,nan,nan,nan,1,nan
2,93,41297.42,41310.83,51.00,nan,157C8653CA,41269.87,41417.80,nan,cancelled,...,NaN,nan,0,nan,0,nan,nan,nan,1,nan
3,94,41291.54,41293.83,45.00,nan,C48194AA4F,41269.91,41417.80,nan,cancelled,...,NaN,nan,0,nan,0,nan,nan,nan,1,nan
4,95,41290.98,41291.79,55.00,nan,AABEB48B94,41269.94,41702.08,nan,finished,...,NaN,nan,0,nan,0,nan,nan,nan,1,nan


In [5]:
np.sum(df.isnull())

id                                                 0
pickup                                             0
dropoff                                            0
user_id                                            1
status                                        462564
confirmation_token                                 0
created_at                                         0
updated_at                                         0
booked_price                                  439186
current_state                                      0
booked_daily_rate                             439186
confirmed                                          0
reservation_type_id                                0
pickup_location_id                                 0
dropoff_location_id                                0
rental_agreement_id                           193945
promo_code_id                                 265766
actual_pickup                                 193823
actual_dropoff                                

In [6]:
df["booking_application"].unique()

array([nan, 'web', 'iphone-appstore', 'android', 'kiosk',
       'iphone-concierge', 'gds', 'iphone-debug', 'web-desktop',
       'web-mobile', 'web-tablet'], dtype=object)

In [7]:
def get_datetime(series):
    return pd.to_datetime('1899-12-30') + pd.to_timedelta(series, 'D')

df["pickup"] = get_datetime(df["pickup"])
df["dropoff"] = get_datetime(df["dropoff"])
df["created_at"] = get_datetime(df["created_at"])
df["updated_at"] = get_datetime(df["updated_at"])

In [8]:
print(df["created_at"].min())
print(df["created_at"].max())

2012-12-26 19:31:35.904000
2017-12-14 19:02:15.935999999


In [9]:
df["current_state"].value_counts()

finished                      268088
cancelled                     162600
no_showed                      23013
booked                          6172
payment_declined_cancelled      2065
started                          621
pending_agreement                  5
Name: current_state, dtype: int64

In [10]:
mask = df["pickup"] > df["created_at"].max()
df_booked = df[df["current_state"] == "booked"]

In [11]:
df_booked.head()

,id,pickup,dropoff,user_id,status,confirmation_token,created_at,updated_at,booked_price,current_state,...,publisher_id,cancelled_at,local_rental,booked_by_id,awards_referral_bonus,extole_conversion_id,potential_local_rental,claimed_at,customer_modifiable,credit_card_id
347533,358808,2017-12-28 16:59:59.712,2018-01-02 17:30:00.288,374628.00,nan,28E9FED670,2017-03-07 07:32:45.024,2017-08-16 21:19:28.128,nan,booked,...,NaN,nan,0,374628.00,0,nan,nan,nan,1,nan
358862,370138,2018-04-20 18:29:59.712,2018-04-23 19:00:00.288,201849.00,nan,B003C40F47,2017-04-05 09:27:33.696,2017-11-02 14:48:40.896,nan,booked,...,NaN,nan,0,nan,0,6405450000000000000.00,nan,nan,1,nan
391915,403193,2017-12-17 22:00:00.288,2017-12-19 01:00:00.288,251806.00,nan,EF7EBFC9DC,2017-06-27 02:46:20.928,2017-10-08 20:44:19.104,nan,booked,...,NaN,nan,0,nan,0,nan,nan,nan,1,nan
392235,403513,2017-12-15 16:59:59.712,2017-12-19 16:59:59.712,413056.00,nan,0115DEE903,2017-06-27 21:47:52.800,2017-06-27 22:00:02.880,nan,booked,...,NaN,nan,0,413056.00,0,nan,nan,nan,1,nan
393171,404449,2017-12-16 15:29:59.712,2017-12-22 21:00:00.000,62675.00,nan,78CAF730D6,2017-06-29 22:38:22.848,2017-12-11 15:09:14.688,nan,booked,...,NaN,nan,0,nan,0,nan,1.00,nan,1,nan


In [12]:
df = df[df["current_state"].isin(["cancelled", "finished"])]

In [13]:
df["current_state"] = df["current_state"].map({"cancelled": 1, "finished": 0})

In [14]:
df["time_to_pickup"] = (df["pickup"] - df["created_at"]).dt.total_seconds() / 86400

In [15]:
df["used_promo"] = (df["promo_code_id"].notnull()).astype(int)
df["same_location"] = (df["pickup_location_id"] == df["dropoff_location_id"]).astype(int)

In [16]:
df["same_location"].sum() / df["same_location"].count()

0.9999930344007727

In [17]:
# df.describe().T

In [18]:
# df.corr().T

In [19]:
df.groupby("current_state").mean().T

current_state,0,1
id,243490.29,216578.76
user_id,195434.73,189254.30
status,nan,nan
booked_price,217.03,232.09
booked_daily_rate,140.78,140.54
confirmed,0.00,0.00
reservation_type_id,2.83,2.46
pickup_location_id,7.91,8.54
dropoff_location_id,7.91,8.54
rental_agreement_id,22.56,12.83


In [20]:
df["pickup_dow"] = df["pickup"].dt.dayofweek
df["pickup_month"] = df["pickup"].dt.month
df["pickup_year"] = df["pickup"].dt.year

In [21]:
df.groupby(["current_state", "pickup_dow"]).mean()

id   user_id  status  booked_price  \
current_state pickup_dow                                             
0             0          238839.56 183282.89     nan        226.30   
              1          239145.07 183295.99     nan        194.63   
              2          244216.69 191947.14     nan        206.10   
              3          245057.63 199024.14     nan        214.58   
              4          244340.79 203794.48     nan        232.47   
              5          246323.19 207325.19     nan        232.43   
              6          250426.02 207660.03     nan        256.33   
1             0          214221.04 185545.59     nan        236.48   
              1          214086.51 184408.79     nan        215.37   
              2          215534.15 187722.02     nan        212.47   
              3          218709.54 191247.21     nan        232.96   
              4          214604.45 188314.29     nan        244.49   
              5          217062.20 190486.56     nan        241.20   
              6          224575.44 199704.73     nan        271.83   

                          booked_daily_rate  confirmed  reservation_type_id  \
current_state pickup_dow                                                      
0             0                      136.78       0.00                 2.77   
              1                      138.68       0.00                 2.78   
              2                      139.32       0.00                 2.83   
              3                      140.87       0.00                 2.84   
              4                      142.29       0.00                 2.85   
              5                      151.81       0.00                 2.87   
              6                      142.25       0.00                 2.88   
1             0                      137.57       0.00                 2.42   
              1                      139.83       0.00                 2.42   
              2                      139.17       0.00                 2.44   
              3                      138.68       0.00                 2.47   
              4                      141.56       0.00                 2.45   
              5                      147.01       0.00                 2.49   
              6                      143.55       0.00                 2.52   

                          pickup_location_id  dropoff_location_id  \
current_state pickup_dow                                            
0             0                         7.57                 7.57   
              1                         7.47                 7.47   
              2                         7.64                 7.64   
              3                         7.87                 7.87   
              4                         8.10                 8.10   
              5                         8.59                 8.59   
              6                         8.67                 8.67   
1             0                         8.61                 8.61   
              1                         8.25                 8.25   
              2                         8.21                 8.21   
              3                         8.44                 8.44   
              4                         8.57                 8.57   
              5                         8.71                 8.71   
              6                         9.20                 9.20   

                          rental_agreement_id     ...       \
current_state pickup_dow                          ...        
0             0                         22.35     ...        
              1                         22.34     ...        
              2                         22.55     ...        
              3                         22.64     ...        
              4                         22.66     ...        
              5                         22.69     ...        
              6                         22.84

In [22]:
df.groupby("pickup_dow").mean()

,id,user_id,status,booked_price,current_state,booked_daily_rate,confirmed,reservation_type_id,pickup_location_id,dropoff_location_id,...,extole_conversion_id,potential_local_rental,claimed_at,customer_modifiable,credit_card_id,time_to_pickup,used_promo,same_location,pickup_month,pickup_year
pickup_dow,,,,,,,,,,,,,,,,,,,,,
0,230726.52,184028.56,nan,230.68,0.33,137.12,0.00,2.65,7.91,7.91,...,6247810630323681280.00,0.40,42873.01,1.00,142520.51,13.59,0.40,1.00,6.70,2015.71
1,231159.86,183650.60,nan,202.98,0.32,139.14,0.00,2.66,7.72,7.72,...,6248237840585600000.00,0.38,42878.23,1.00,142213.48,13.49,0.38,1.00,6.73,2015.71
2,234390.14,190499.63,nan,208.79,0.34,139.26,0.00,2.69,7.84,7.84,...,6249645923893154816.00,0.37,42893.06,1.00,146052.57,16.73,0.40,1.00,6.80,2015.74
3,235027.47,196063.63,nan,222.83,0.38,139.89,0.00,2.70,8.09,8.09,...,6252634372961763328.00,0.39,42903.86,1.00,150263.92,21.35,0.44,1.00,6.82,2015.76
4,231657.61,197191.85,nan,238.60,0.43,141.92,0.00,2.68,8.31,8.31,...,6252032472487173120.00,0.48,42907.37,1.00,154092.36,22.96,0.47,1.00,6.74,2015.74
5,233320.83,199842.81,nan,236.74,0.44,149.45,0.00,2.70,8.65,8.65,...,6261690945303214080.00,0.50,42903.75,1.00,156555.31,21.73,0.47,1.00,6.76,2015.74
6,239909.61,204423.69,nan,263.73,0.41,142.87,0.00,2.73,8.88,8.88,...,6254190777222006784.00,0.48,42878.87,1.00,154501.54,18.23,0.45,1.00,6.74,2015.80


In [23]:
df["weekend_pickup"] = (df["pickup_dow"].isin([4, 5, 6])).astype(int)

In [24]:
df.groupby("pickup_month").mean()

,id,user_id,status,booked_price,current_state,booked_daily_rate,confirmed,reservation_type_id,pickup_location_id,dropoff_location_id,...,potential_local_rental,claimed_at,customer_modifiable,credit_card_id,time_to_pickup,used_promo,same_location,pickup_dow,pickup_year,weekend_pickup
pickup_month,,,,,,,,,,,,,,,,,,,,,
1,199859.02,168206.98,nan,245.03,0.42,137.79,0.00,2.19,7.52,7.52,...,0.39,42698.45,1.00,138779.73,15.59,0.38,1.00,2.85,2015.94,0.41
2,201481.07,167017.36,nan,226.98,0.39,137.76,0.00,2.73,7.31,7.31,...,0.33,42706.49,1.00,147617.75,17.68,0.40,1.00,2.85,2015.89,0.41
3,206870.00,171899.47,nan,227.50,0.36,139.83,0.00,2.84,7.30,7.30,...,0.36,42753.28,1.00,148332.07,18.11,0.41,1.00,2.78,2015.85,0.38
4,214341.96,176738.02,nan,210.71,0.36,141.25,0.00,2.84,7.26,7.26,...,0.38,42803.21,1.00,121310.36,17.53,0.44,1.00,2.86,2015.82,0.41
5,219657.74,179925.88,nan,205.06,0.36,136.02,0.00,2.80,7.54,7.54,...,0.42,42841.29,1.00,128837.05,17.79,0.44,1.00,2.86,2015.78,0.41
6,227983.91,189976.57,nan,223.30,0.37,142.52,0.00,2.78,7.93,7.93,...,0.41,42870.54,1.00,136732.69,17.69,0.46,1.00,2.73,2015.76,0.37
7,233911.64,200064.19,nan,256.36,0.39,147.04,0.00,2.71,8.39,8.39,...,0.46,42896.86,1.00,146757.60,20.40,0.45,1.00,2.86,2015.74,0.41
8,245871.82,206799.48,nan,247.46,0.37,154.55,0.00,2.72,8.88,8.88,...,0.49,42942.81,1.00,154601.64,19.08,0.44,1.00,2.75,2015.73,0.38
9,256490.42,211234.26,nan,205.16,0.35,137.34,0.00,2.75,8.66,8.66,...,0.49,42945.48,1.00,158972.11,18.76,0.42,1.00,2.89,2015.73,0.42


In [25]:
df["dec_jan_pickup"] = (df["pickup_dow"].isin([1, 2])).astype(int)

In [26]:
df.groupby("pickup_year").mean()

,id,user_id,status,booked_price,current_state,booked_daily_rate,confirmed,reservation_type_id,pickup_location_id,dropoff_location_id,...,claimed_at,customer_modifiable,credit_card_id,time_to_pickup,used_promo,same_location,pickup_dow,pickup_month,weekend_pickup,dec_jan_pickup
pickup_year,,,,,,,,,,,,,,,,,,,,,
2013,8877.32,8255.29,nan,236.12,0.53,160.30,0.00,1.00,2.54,2.54,...,nan,1.00,nan,12.36,0.36,1.00,2.81,8.52,0.40,0.31
2014,40949.75,36547.74,nan,259.35,0.45,153.29,0.00,1.00,4.39,4.39,...,nan,1.00,127293.00,15.49,0.56,1.00,2.84,7.09,0.41,0.30
2015,124796.19,116329.92,nan,229.52,0.42,128.40,0.00,1.00,6.52,6.52,...,42360.35,1.00,nan,18.11,0.38,1.00,2.78,7.11,0.38,0.31
2016,254858.47,210167.18,nan,216.39,0.33,139.21,0.00,3.65,8.40,8.40,...,42632.06,1.00,103061.07,18.36,0.35,1.00,2.82,6.41,0.40,0.30
2017,397150.56,316422.06,nan,221.28,0.34,142.17,0.00,4.00,11.19,11.19,...,42944.40,1.00,183982.69,20.06,0.52,1.00,2.89,6.52,0.41,0.29
2018,456592.44,379265.88,nan,230.80,1.00,127.64,0.00,4.00,14.77,14.77,...,43066.94,1.00,nan,94.26,0.35,1.00,3.14,1.80,0.49,0.24


In [27]:
df["awards_referral_bonus"].value_counts()

0    418399
1     12289
Name: awards_referral_bonus, dtype: int64

In [28]:
df["local_rental"].value_counts()

0    418682
1     12006
Name: local_rental, dtype: int64

In [29]:
df["created_as_guest"].value_counts()

0    341658
1     89030
Name: created_as_guest, dtype: int64

In [45]:
cols = ["user_id", "current_state", "time_to_pickup", "reservation_frequency", "used_promo"]
# cols = ["user_id", "current_state"]
df_model = df[cols]
df_model = pd.get_dummies(df_model)

In [47]:
df_users_model = df_users[["sign_in_count", "is_gds_user", "id"]].set_index("id")

In [48]:
df_model = df_model.join(df_users_model, how="left", on="user_id")

In [49]:
df_model.drop("user_id", axis=1, inplace=True)
df_model.dropna(inplace=True)

In [50]:
y = df_model.pop("current_state").values
X = df_model.values
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [51]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.77453603076408628

In [52]:
rf = RandomForestClassifier(max_depth=5, n_estimators=50)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.80205837002359326

In [143]:
cols = ["user_id", "current_state", "time_to_pickup", "reservation_frequency", "used_promo",
       "promo_code_id"]
df_model = df[cols]
df_model = pd.get_dummies(df_model)

In [144]:
df_codes = df_promo_codes[["id", "promotion_id"]].set_index("id")

In [145]:
df_promo_model = df_codes.join(df_promos[["value", "id"]].set_index("id"), how="left")

In [146]:
df_promo_model.drop("promotion_id", axis=1, inplace=True)

In [147]:
df_model = df_model.join(df_promo_model, how="left", on="promo_code_id")
df_model.drop("promo_code_id", axis=1, inplace=True)
df_model.value.fillna(0, inplace=True)

In [148]:
df_users_model = df_users[["sign_in_count", "is_gds_user", "id"]].set_index("id")
df_model = df_model.join(df_users_model, how="left", on="user_id")
df_model.drop("user_id", axis=1, inplace=True)

In [149]:
df_model.dropna(inplace=True)

In [150]:
y = df_model.pop("current_state").values
X = df_model.values
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [151]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.77757342696316112

In [152]:
rf = RandomForestClassifier(max_depth=5, n_estimators=50)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.80719500640918462